# Pretraining BERT from scratch 

In [14]:
import torch
from tqdm.auto import tqdm

In [3]:
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" > /dev/null


In [5]:
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" -o /home/sol3sts/wiki_20190620_small.txt

awk: fatal: cannot open file `./cookie' for reading (그런 파일이나 디렉터리가 없습니다)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1323k  100 1323k    0     0  1065k      0  0:00:01  0:00:01 --:--:-- 3987k


In [11]:
# tokenizer 직접 학습 

from tokenizers import BertWordPieceTokenizer

wp_tokenizer = BertWordPieceTokenizer(
    clean_text = True,
    handle_chinese_chars = True,
    strip_accents = False,
    lowercase = False)

wp_tokenizer.train(
    files = '/home/sol3sts/wiki_20190620_small.txt',
            vocab_size = 20000,
            min_frequency = 2,
            show_progress = True,
            special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
            wordpieces_prefix = "##"
)

wp_tokenizer.save_model('/home/sol3sts/')

['/home/sol3sts/vocab.txt']

In [44]:
# tokenizer 정의하기 

from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast(
    vocab_file = '/home/sol3sts/vocab.txt', # 학습한 wp-tknzer vocab
    max_len=128,
    do_lower_case=False
)

tokenizer.tokenize("나는 서광욱이다.")

['나는', '서', '##광', '##욱', '##이다', '.']

In [45]:
# model config setting 

from transformers import BertConfig, BertForPreTraining

config = BertConfig(
    vocab_size = 20000,
    max_position_embeddings=128) # max input dim (max len)


model = BertForPreTraining(config=config)

## Dataset & DataLoader

In [46]:
# dataset & dataloader

from torch.utils.data.dataset import Dataset
from transformers.tokenization_utils import PreTrainedTokenizer
from typing import Dict,List,Optional

import os
import json
import pickle
import random
import time
import warnings

from filelock import FileLock

from transformers.utils import logging

logger = logging.get_logger(__name__)


In [54]:
# dataset class 정의 
class TextDatasetForNextSentencePrediction(Dataset):
    """
    This will be superseded by a framework-agnostic approach soon.
    """

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        file_path: str,
        block_size: int,
        overwrite_cache=False,
        short_seq_probability=0.1,
        nsp_probability=0.5,
    ):
        
        
        # train data caching 
        assert os.path.isfile(file_path), f"Input file path {file_path} not found"
        
        self.block_size = block_size - tokenizer.num_special_tokens_to_add(pair=True)
        self.short_seq_probability = short_seq_probability
        self.nsp_probability = nsp_probability

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            directory,
            f"cached_nsp_{tokenizer.__class__.__name__}_{block_size}_{filename}",
        )

        
        
        
        
        self.tokenizer = tokenizer

        
        lock_path = cached_features_file + ".lock"

        # Input file format:
        # (1) One sentence per line. These should ideally be actual sentences, not
        # entire paragraphs or arbitrary spans of text. (Because we use the
        # sentence boundaries for the "next sentence prediction" task).
        # (2) Blank lines between documents. Document boundaries are needed so
        # that the "next sentence prediction" task doesn't span between documents.
        #
        # Example:
        # I am very happy.
        # Here is the second sentence.
        #
        # A new document.

        with FileLock(lock_path):
            if os.path.exists(cached_features_file) and not overwrite_cache:
                start = time.time()
                with open(cached_features_file, "rb") as handle:
                    self.examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )
            else: # if there's no cache 
                logger.info(f"Creating features from dataset file at {directory}")

                
                # make dataset 
                self.documents = [[]] # document 단위의 학습 
                with open(file_path, encoding="utf-8") as f:
                    while True: # 문장읽기 
                        line = f.readline()
                        if not line:
                            break
                        line = line.strip() # strip 

                        # 문단단위 데이터 저장 (이중공백일경우 나왔던 문장들을 하나의 문서로 저장)
                        if not line and len(self.documents[-1]) != 0:
                            self.documents.append([])
                        tokens = tokenizer.tokenize(line)
                        tokens = tokenizer.convert_tokens_to_ids(tokens)
                        if tokens:
                            self.documents[-1].append(tokens)
                            
                # 코퍼스 전체를 읽고 문서데이터를 생성 
                logger.info(f"Creating examples from {len(self.documents)} documents.")
                
                # input을 위한 data로 변경 
                self.examples = []
                for doc_index, document in enumerate(self.documents):
                    self.create_examples_from_document(document, doc_index, block_size)

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    f"Saving features into cached file {cached_features_file} [took {time.time() - start:.3f} s]"
                )

                
                
    def create_examples_from_document(self, document: List[List[int]], doc_index: int, block_size: int):
        """Creates examples for a single document."""
        
        # 문장 앞, 뒤에 CLS, SEP 토큰이 부착되기 때문에 지정한 size에서 2만큼 빼준다. 
        # max_len (max_embedding) = 128 이라면 126 token만 가져오는것 
        max_num_tokens = block_size - self.tokenizer.num_special_tokens_to_add(pair=True)
        
        
        # computation낭비를 막기위해 short_seq_prob 만큼의 데이터에서는 2-126사이의 random값으로 데이터 만듬 
        target_seq_length = max_num_tokens
        if random.random() < self.short_seq_probability:
            target_seq_length = random.randint(2, max_num_tokens)

        current_chunk = []  # a buffer stored current working segments
        current_length = 0
        i = 0

        # 데이터구축의 최소단위 : document
        # sent1 [SEP] sent2 / sent1 + sent2 [SEP] sent3 + sent4 - 126size가 남는다면
        while i < len(document):
            segment = document[i]
            current_chunk.append(segment)
            current_length += len(segment)
            if i == len(document) - 1 or current_length >= target_seq_length:
                if current_chunk:
                    # `a_end` is how many segments from `current_chunk` go into the `A`
                    # (first) sentence.
                    a_end = 1
                    if len(current_chunk) >= 2:
                        # 두문장이상이 첫문장으로 input시 랜덤 cut
                        a_end = random.randint(1, len(current_chunk) - 1)

                    tokens_a = []
                    for j in range(a_end):
                        tokens_a.extend(current_chunk[j])

                    tokens_b = []

                    if len(current_chunk) == 1 or random.random() < self.nsp_probability:
                        is_random_next = True
                        target_b_length = target_seq_length - len(tokens_a)

                        # This should rarely go for more than one iteration for large
                        # corpora. However, just to be careful, we try to make sure that
                        # the random document is not the same as the document
                        # we're processing.
                        for _ in range(10):
                            random_document_index = random.randint(0, len(self.documents) - 1)
                            if random_document_index != doc_index:
                                break

                        random_document = self.documents[random_document_index]
                        random_start = random.randint(0, len(random_document) - 1)
                        for j in range(random_start, len(random_document)):
                            tokens_b.extend(random_document[j])
                            if len(tokens_b) >= target_b_length:
                                break
                        # We didn't actually use these segments so we "put them back" so
                        # they don't go to waste.
                        num_unused_segments = len(current_chunk) - a_end
                        i -= num_unused_segments
                    # Actual next
                    else:
                        is_random_next = False
                        for j in range(a_end, len(current_chunk)):
                            tokens_b.extend(current_chunk[j])
                    
                    # max_len 넘을경우 truncation
                    def truncate_seq_pair(tokens_a, tokens_b, max_num_tokens):
                        while True:
                            total_length = len(tokens_a) + len(tokens_b)
                            if total_length <= max_num_tokens:
                                break
                            
                            trunc_tokens = tokens_a if len(tokens_a) > len(tokens_b) else tokens_b
                            
                            assert len(trunc_tokens) >=1
                            
                            # bias를 피하기위해 끝과시작에서 랜덤하게 truncate
                            if random.random() < 0.5:
                                del trunc_tokens[0]
                            else:
                                trunc_tokens.pop()
                    
                    truncate_seq_pair(tokens_a, tokens_b, max_num_tokens)
                    
                    assert len(tokens_a)>=1
                    assert len(tokens_b)>=1
                    
                    
                    # add special tokens
                    input_ids = self.tokenizer.build_inputs_with_special_tokens(tokens_a, tokens_b)
                    # add token type ids, 0 for sentence a, 1 for sentence b
                    token_type_ids = self.tokenizer.create_token_type_ids_from_sequences(tokens_a, tokens_b)

                    example = {
                        "input_ids": torch.tensor(input_ids, dtype=torch.long),
                        "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
                        "next_sentence_label": torch.tensor(1 if is_random_next else 0, dtype=torch.long),
                    }

                    self.examples.append(example)

                current_chunk = []
                current_length = 0

            i += 1

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

In [55]:
dataset = TextDatasetForNextSentencePrediction(
        tokenizer = tokenizer,
        file_path = '/home/sol3sts/wiki_20190620_small.txt',
        block_size = 128,
        overwrite_cache = False,
        short_seq_probability = 0.1,
        nsp_probability = 0.5
        
)

In [53]:
# data_collator 

data_collator = DataCollatorForLanguageModeling(
    tokenizer = tokenizer,
    mlm = True, # masking 
    mlm_probability = 0.15
)

In [65]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = '/home/sol3sts/model_output',
    overwrite_output_dir = True,
    num_train_epochs = 10,
    per_device_train_batch_size = 32,
    save_steps = 1000,
    save_total_limit = 2,
    logging_steps=100
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [66]:
trainer = Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = dataset
)

In [64]:
model

BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(20000, 768, padding_idx=0)
      (position_embeddings): Embedding(128, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [88]:
from transformers import BertForMaskedLM, pipeline

my_model = BertForMaskedLM.from_pretrained('model_output')
nlp_fill = pipeline('fill-mask', top_k=5, model=my_model, tokenizer=tokenizer)
nlp_fill('이순신은 [MASK] 중기의 무신이다.')

OSError: model_output does not appear to have a file named config.json. Checkout 'https://huggingface.co/model_output/None' for available files.

In [83]:
tokenizer.mask_token

'[MASK]'